## Import Lybs

In [1]:
import os
import cv2
import albumentations as A
import numpy as np

## Image augmentation

In [3]:
def load_yolo_annotations(file_path):
    """Load YOLO annotations from a file."""
    with open(file_path, "r") as file:
        annotations = []
        for line in file:
            data = line.strip().split()
            label = int(data[0])  # Class label
            bbox = list(map(float, data[1:]))  # YOLO format: [x_center, y_center, width, height]
            annotations.append((label, bbox))
    return annotations

def save_yolo_annotations(file_path, annotations):
    """Save YOLO annotations to a file."""
    with open(file_path, "w") as file:
        for label, bbox in annotations:
            bbox_str = " ".join(map(str, bbox))
            file.write(f"{label} {bbox_str}\n")

def clip_bboxes(bboxes):
    """Clip bounding boxes to stay within [0, 1] range."""
    clipped_bboxes = []
    for bbox in bboxes:
        x_center, y_center, width, height = bbox
        x_center = max(0, min(1, x_center))
        y_center = max(0, min(1, y_center))
        width = max(0, min(1, width))
        height = max(0, min(1, height))
        clipped_bboxes.append([x_center, y_center, width, height])
    return clipped_bboxes

def augment_images_and_annotations(image_folder, annotation_folder):

    transformations = [
        (A.Compose([A.HorizontalFlip(p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "horizontalflip"),
        (A.Compose([A.GaussianBlur(blur_limit=(3, 7), p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "gaussianblur"),
        (A.Compose([A.RandomBrightnessContrast(p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "brightnesscontrast"),
        (A.Compose([A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "huesaturationvalue"),
        (A.Compose([A.CoarseDropout(max_holes=8, max_height=32, max_width=32, min_holes=1, min_height=16, min_width=16, fill_value=0, p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "coarsedropout"),
        (A.Compose([A.Perspective(scale=(0.05, 0.1), p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "perspective"),
        (A.Compose([A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "elastictransform"),
        (A.Compose([A.RandomScale(scale_limit=0.2, p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "randomscale")
    ]

   

    # Process each image and its corresponding annotation
    for file_name in os.listdir(image_folder):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Load image
            image_path = os.path.join(image_folder, file_name)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Failed to load image {file_name}, skipping...")
                continue

            # Load corresponding annotation
            base_name, ext = os.path.splitext(file_name)
            annotation_path = os.path.join(annotation_folder, f"{base_name}.txt")
            if not os.path.exists(annotation_path):
                print(f"Annotation file for {file_name} not found, skipping...")
                continue

            annotations = load_yolo_annotations(annotation_path)
            bboxes = [bbox for _, bbox in annotations]
            class_labels = [label for label, _ in annotations]

            # Apply each augmentation
            for transform, aug_name in transformations:
                try:
                    augmented = transform(
                        image=image,
                        bboxes=bboxes,
                        class_labels=class_labels
                    )
                    augmented_image = augmented["image"]
                    augmented_bboxes = augmented["bboxes"]

                    # Clip bounding boxes
                    augmented_bboxes = clip_bboxes(augmented_bboxes)

                    # Save augmented image
                    new_file_name = f"{base_name}_{aug_name}{ext}"
                    save_path_image = os.path.join(image_folder, new_file_name)
                    cv2.imwrite(save_path_image, augmented_image)

                    # Save augmented annotation
                    new_annotation_file = f"{base_name}_{aug_name}.txt"
                    save_path_annotation = os.path.join(annotation_folder, new_annotation_file)
                    augmented_annotations = [
                        (class_labels[i], augmented_bboxes[i])
                        for i in range(len(class_labels))
                    ]
                    save_yolo_annotations(save_path_annotation, augmented_annotations)

                    # print(f"Saved augmented image: {new_file_name} and annotation: {new_annotation_file}")
                except Exception as e:
                    print(f"Failed to augment {file_name} with {aug_name}: {e}")

    print("Data augmentation completed!")


In [4]:
# Specify folder paths
image_folder = "dataset\\images"
annotation_folder = "dataset\\labels"

# Run augmentation
augment_images_and_annotations(image_folder, annotation_folder)

c:\Users\Hashan\anaconda3\envs\agm-image\lib\site-packages\albumentations\core\composition.py:243: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
C:\Users\Hashan\AppData\Local\Temp\ipykernel_2848\935390809.py:40: UserWarning: Argument 'alpha_affine' is not valid and will be ignored.
  (A.Compose([A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=1.0)], bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"])), "elastictransform"),


Data augmentation completed!


## Displya images with their corresponding BBX

In [5]:
import os
import cv2
import matplotlib.pyplot as plt

def read_yolo_label(label_path):
    """
    Reads YOLO format label data from a text file.

    Parameters:
    - label_path: Path to the label file.

    Returns:
    - A list of tuples containing (class_id, x_center, y_center, width, height).
    """
    with open(label_path, 'r') as f:
        labels = []
        for line in f.readlines():
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            labels.append((class_id, x_center, y_center, width, height))
        return labels

def draw_bounding_boxes(image, labels, class_names):
    """
    Draws bounding boxes on an image.

    Parameters:
    - image: The input image (numpy array).
    - labels: A list of YOLO labels [(class_id, x_center, y_center, width, height), ...].
    - class_names: List of class names corresponding to class IDs.

    Returns:
    - The image with bounding boxes drawn on it.
    """
    h, w = image.shape[:2]
    for label in labels:
        class_id, x_center, y_center, width, height = label
        x1 = int((x_center - width / 2) * w)
        y1 = int((y_center - height / 2) * h)
        x2 = int((x_center + width / 2) * w)
        y2 = int((y_center + height / 2) * h)
        
        # Draw the rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        # Put the class label
        label_text = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
        cv2.putText(image, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 3)
    return image

def display_images_with_labels(image_folder, label_folder, class_names):
    """
    Displays images with bounding boxes using YOLO labels.

    Parameters:
    - image_folder: Path to the folder containing images.
    - label_folder: Path to the folder containing YOLO label files.
    - class_names: List of class names corresponding to class IDs.
    """
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', 'jpeg'))])
    
    for image_file in image_files:
        # Get corresponding label file
        label_file = os.path.join(label_folder, os.path.splitext(image_file)[0] + '.txt')
        image_path = os.path.join(image_folder, image_file)
        
        # Read image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Read labels
        if os.path.exists(label_file):
            labels = read_yolo_label(label_file)
        else:
            labels = []
        
        # Draw bounding boxes
        image_with_boxes = draw_bounding_boxes(image, labels, class_names)
        # print(labels)
        # Display image
        plt.figure(figsize=(8, 8))
        plt.imshow(image_with_boxes)
        plt.axis('off')
        plt.title(image_file)
        plt.show()

# Example Usage



In [ ]:
image_folder = "dataset\\images"  # Replace with your image folder path
label_folder = "dataset\\labels"  # Replace with your label folder path
class_names = ["bulathwita","dodol","hakuru","kiribath","pani_walalu"]  # Replace with your class names

display_images_with_labels(image_folder, label_folder, class_names)

## Split Data set in to train and val

In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_dataset(images_folder, annotations_folder, output_folder, train_ratio=0.7):
    # Ensure output directories exist
    train_images_folder = os.path.join(output_folder, "train", "images")
    val_images_folder = os.path.join(output_folder, "val", "images")
    train_annotations_folder = os.path.join(output_folder, "train", "annotations")
    val_annotations_folder = os.path.join(output_folder, "val", "annotations")
    
    os.makedirs(train_images_folder, exist_ok=True)
    os.makedirs(val_images_folder, exist_ok=True)
    os.makedirs(train_annotations_folder, exist_ok=True)
    os.makedirs(val_annotations_folder, exist_ok=True)

    # Get a list of image files and corresponding annotation files
    image_files = sorted([f for f in os.listdir(images_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])
    annotation_files = sorted([f for f in os.listdir(annotations_folder) if f.endswith('.txt')])
    
    # Ensure image files have corresponding annotation files
    paired_data = [
        (os.path.join(images_folder, img), os.path.join(annotations_folder, img.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')))
        for img in image_files if os.path.exists(os.path.join(annotations_folder, img.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt')))
    ]

    # Split into train and validation sets
    train_data, val_data = train_test_split(paired_data, train_size=train_ratio, random_state=42)
    
    # Copy files to respective folders
    for img_path, ann_path in train_data:
        shutil.copy(img_path, train_images_folder)
        shutil.copy(ann_path, train_annotations_folder)

    for img_path, ann_path in val_data:
        shutil.copy(img_path, val_images_folder)
        shutil.copy(ann_path, val_annotations_folder)

    print(f"Data successfully split into training and validation sets.")
    print(f"Training set: {len(train_data)} images, Validation set: {len(val_data)} images.")




In [9]:
# Usage:
images_folder = "dataset\\images"
annotations_folder = "dataset\\labels"
output_folder = ""

split_dataset(images_folder, annotations_folder, output_folder)

Data successfully split into training and validation sets.
Training set: 327 images, Validation set: 141 images.
